In [1]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg texlive texlive-latex-extra texlive-fonts-extra texlive-latex-recommended texlive-science tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://developer.

In [1]:
from manim import * 

class PointMovingOnShapes(Scene):
    def construct(self):
        square = Square(color=BLUE) # Create a square
        square.flip(RIGHT) # Flip the square to the right
        square.rotate(-3 * TAU / 8) # Rotate the square -3/8 * 2*PI 

         # Play the animation of a square growing from the center
        self.play(GrowFromCenter(square))

In [3]:
!manim -p -ql start.py PointMovingOnShapes

Manim Community v0.5.0
[04/16/21 12:33:48] INFO     Animation 0 : Partial      ]8;id=1618576428.9810872-403958;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:397
                             movie file written in {'/c                         
                             ontent/media/videos/start/                         
                             480p15/partial_movie_files                         
                             /PointMovingOnShapes/31637                         
                             82288_625883850_3881855864                         
                             .mp4'}                                             
[04/16/21 12:33:49] INFO                                ]8;id=1618576429.0689483-794772;file:///usr/local/lib/python3.7/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:579
                             File ready at /content/med                         
        

In [ ]:
from keras import Sequential
from keras.layers import SimpleRNN, Dense, Input, Embedding, LSTM
from keras.layers.experimental.preprocessing import TextVectorization

#Recurrent Neurons

A neuron which at time step t takes as input the output from the previous time step t-1 along with the inputs. It sends its outputs back to itself for each next step.

It has two sets of weights:
* w<sub>x</sub> for the inputs x<sub>(t)</sub>, and 
* w<sub>y</sub> for the outputs of the previous time step y<sub>(t-1)</sub>

Then the output of a recurrent neuron is:

y<sub>(t)</sub> = ϕ(x<sub>(t)</sub><sup>T</sup> . w<sub>x</sub> + y<sub>(t-1)</sub> <sup>T</sup> . w<sub>y</sub> + b)

The output for the whole layer of recurrent neurons will be:

Y<sub>(t)</sub> = ϕ(X<sub>(t)</sub> . W<sub>x</sub> + Y<sub>(t-1)</sub> . W<sub>y</sub> + b)

where X, Y and W are matrices of inputs, outputs and weights for the whole mini-batch, and b is the bias vector of size = the number of neurons.

# Memory

Now the output at t is a function of the inputs and the previous outputs:

Y<sub>(t)</sub> = f(X<sub>(t)</sub>, Y<sub>(t-1)</sub>)

Since Y<sub>(t-1)</sub> is a function of X<sub>(t-1)</sub> and Y<sub>(t-2)</sub>,

Y<sub>(t)</sub> = f(X<sub>(t)</sub>, g(X<sub>(t-1)</sub>, Y<sub>(t-2)</sub>))

and so on:

Y<sub>(t)</sub> = f(X<sub>(t)</sub>, g(X<sub>(t-1)</sub>, h(X<sub>(t-2)</sub>, Y<sub>(t-3)</sub>)))

Hence, Y<sub>(t)</sub> is a function of all the inputs since time t = 0.

This gives the output of a recurrent neuron at t, memory of all the inputs from previous time steps.

# Simple RNN layer

In [ ]:
moedl = Sequential([
                    SimpleRNN(10, #10 neurons
                              input_shape=(50, 1), #processes 50x1 sequences
                              return_dequences=True #needs to be used when stacking multiple RNN layers together
                              )
])

# Input and Output Sequences

## Sequence-to-Sequence Networks

Feed a network a sequence of inputs and produce a sequence of outputs.

Application: Stock-price prediction.

In [ ]:
model = Sequential([
                    SimpleRNN(64, input_shape=(50, 1), return_sequences=True),
                    SimpleRNN(32, return_sequences=True),
                    SimpleRNN(16), #doesn't need to return sequence since it's not followed by an RNN layer

                    Dense(8, activation='tanh'),
                    Dense(1, activation='linear') #linear activation for the regression task of predicting stock prices
])

In [ ]:
model.compile(loss='mse', optimizer='Adam', metrics=['mae'])

## Sequence-to-Vector Networks

Feed a network a sequence of inputs and produce a vector.

Application: Predicting positive/negative sentiment in a tweet.

In [ ]:
import numpy
from keras.datasets import imdb
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [ ]:
model = Sequential([
                    Embedding(5000, #top words
                              32, #embedding vector length
                              input_length=500 #length of the longest input sequence
                              ),
                    LSTM(100),
                    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

## Vector-to-Sequence Networks

Feed a network a vector and produce a sequence of outputs.

Application: Generating captions for an image.

In [ ]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

## Encoder-Decoder Networks

Comprises of:

1. **Encoder:** A sequence-to-vector network:

  Feed a network a sequence of inputs and produce a vector.

2. **Decoder:** A vector-to-sequence network:

  Take the output vectors from the encoder as input, and produce a sequence of outputs.

Application: Machine translation.

In [ ]:
input_sequence = Input(shape=(max_spanish_len,))
embedding = Embedding(input_dim=spanish_vocab, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)

enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])

RNNs are only good for short-term dependencies. Due to the Vanishing Gradient problem, relevant information in a sequences may not be available due to the presence of irrelevant data.

#LSTMs

LSTMs selectively remember patterns for long durations of time.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils